# 18 - RAM Run

https://adventofcode.com/2024/day/18


In [6]:
// Imports

import colors from "../../utils/colors.ts"
import objects from "../../utils/objects.ts"
import strings from "../../utils/strings.ts"
import numbers from "../../utils/numbers.ts"
import plots from "../../utils/plots.ts";
import images from "../../utils/images.ts";
import arrays from "../../utils/arrays.ts";

In [7]:
// Read Input

const file = await Deno.readTextFile("input-base.txt");
const fallingBytes = file.split("\n").map((line) => line.split(","));
fallingBytes

[
  [ "5", "4" ], [ "4", "2" ],
  [ "4", "5" ], [ "3", "0" ],
  [ "2", "1" ], [ "6", "3" ],
  [ "2", "4" ], [ "1", "5" ],
  [ "0", "6" ], [ "3", "3" ],
  [ "2", "6" ], [ "5", "1" ],
  [ "1", "2" ], [ "5", "5" ],
  [ "2", "5" ], [ "6", "5" ],
  [ "1", "4" ], [ "0", "4" ],
  [ "6", "4" ], [ "1", "1" ],
  [ "6", "1" ], [ "1", "0" ],
  [ "0", "5" ], [ "1", "6" ],
  [ "2", "0" ]
]

In [8]:
// Prepare Data

const DIMENSIONS = [7, 7];

In [9]:
// Part 1 - what is the minimum number of steps needed to reach the exit?

const isCorrupted = el => el === '#';
const visitedKey = (i, j) => `${i}-${j}`;
const directions = [[0, 1], [1, 0], [0, -1], [-1, 0]];

const findMinSteps = (area: string[][], startPosition: [], maxIterations = 1000000) => {
    const stack = [{ position: startPosition, path: new Set<string>(), steps: 0 }];

    let minSteps = Number.MAX_SAFE_INTEGER;
    let minPath = null;
    const visited = {}

    while (stack.length > 0) {
        const { position, path, steps } = stack.shift();
        const [i, j] = position;

        if (i < 0 || i >= area.length || j < 0 || j >= area[0].length) {
            continue;
        }

        if (maxIterations-- < 0) break;

        const currentPathKey = visitedKey(i, j);
        if (visited[currentPathKey]) continue;
        visited[currentPathKey] = true;

        path.add(currentPathKey);
        if (i === area.length - 1 && j === area[0].length - 1) {
            // since path is not weighted, first reach will be shortest of all
            minPath = path;
            minSteps = steps;
            break;
        }

        directions.forEach((d) => {
            const next = area[i + d[0]] ? area[i + d[0]][j + d[1]] : null;
            if (next && !isCorrupted(next)) {
                stack.push({ position: [i + d[0], j + d[1]], path: new Set([...path]), steps: steps + 1 });
            }
        });
    }
    return [minSteps, minPath];
}

const currentArea = new Array(DIMENSIONS[0]).fill(0).map(() => new Array(DIMENSIONS[1]).fill('.'));
fallingBytes.slice(0, 12).forEach(([i, j]) => {
    currentArea[i][j] = '#';
});
const [minSteps, minPath] = findMinSteps(currentArea, [0, 0]);
[...minPath].forEach((pathKey, index) => {
    if (index === 0) return;
    const [i, j] = pathKey.split('-');
    currentArea[i][j] = '0';
});
colors.visualize2DArray(currentArea, {
    colors: {
        '.': colors.bgWhite,
        '0': colors.bgGreen,
        '#': colors.bgBlack,
    }
});

[minSteps]

[ 22 ]

In [10]:
// Part 2 - What are the coordinates of the first byte that will prevent the exit from being reachable from your starting position

const part2Area = new Array(DIMENSIONS[0]).fill(0).map(() => new Array(DIMENSIONS[1]).fill('.'));
fallingBytes.slice(0, 12).forEach(([i, j]) => {
    currentArea[i][j] = '#';
});
let lastPath = null
for (let i = 12; i < fallingBytes.length; i++) {
    currentArea[fallingBytes[i][0]][fallingBytes[i][1]] = '#';
    if (lastPath && !lastPath.has(visitedKey(fallingBytes[i][0], fallingBytes[i][1]))) continue;
    const [_, minPath] = findMinSteps(currentArea, [0, 0]);
    if (!minPath) {
        console.log(fallingBytes[i]);
        break;
    }
    else {
        lastPath = minPath;
    }
}

[ "6", "1" ]
